In [1]:

import numpy as np 
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/creditcardfraud/creditcard.csv


In [2]:
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest

print("Loading Spending Data...")
data = pd.read_csv('/kaggle/input/creditcardfraud/creditcard.csv')

print(f"Original Data Size: {data.shape}")
print("Columns: Time, Amount, V1-V28 (Anonymized features)")

data_sample = data.sample(n=50000, random_state=42)

print(" Data Loaded & Sampled.")
print(data_sample.head())

Loading Spending Data...
Original Data Size: (284807, 31)
Columns: Time, Amount, V1-V28 (Anonymized features)
 Data Loaded & Sampled.
            Time         V1        V2         V3        V4         V5  \
43428    41505.0 -16.526507  8.584972 -18.649853  9.505594 -13.793819   
49906    44261.0   0.339812 -2.743745  -0.134070 -1.385729  -1.451413   
29474    35484.0   1.399590 -0.590701   0.168619 -1.029950  -0.539806   
276481  167123.0  -0.432071  1.647895  -1.669361 -0.349504   0.785785   
278846  168473.0   2.014160 -0.137394  -1.015839  0.327269  -0.182179   

              V6         V7        V8        V9  ...       V21       V22  \
43428  -2.832404 -16.701694  7.517344 -8.507059  ...  1.190739 -1.127670   
49906   1.015887  -0.524379  0.224060  0.899746  ... -0.213436 -0.942525   
29474   0.040444  -0.712567  0.002299 -0.971747  ...  0.102398  0.168269   
276481 -0.630647   0.276990  0.586025 -0.484715  ...  0.358932  0.873663   
278846 -0.956571   0.043241 -0.160746  0.363241

In [3]:

features = ['Amount'] + [f'V{i}' for i in range(1, 29)]


X = data_sample[features]


X = X.fillna(0)

print(f" Data Ready for Training.")
print(f"Features selected: {len(features)}")

 Data Ready for Training.
Features selected: 29


In [4]:
print("Training Treasury Monitor (Isolation Forest)...")
iso_model = IsolationForest(n_estimators=100, contamination=0.02, random_state=42, n_jobs=-1)


iso_model.fit(X)

data_sample['anomaly_score'] = iso_model.decision_function(X)
data_sample['prediction'] = iso_model.predict(X) # Returns -1 for Fraud, 1 for Normal

n_anomalies = (data_sample['prediction'] == -1).sum()
print(" Training Complete!")
print(f"Model found {n_anomalies} anomalies in the training set.")

Training Treasury Monitor (Isolation Forest)...


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


 Training Complete!
Model found 1000 anomalies in the training set.


In [5]:
def check_spending_risk(transaction_data):
    """
    Input: A dictionary of transaction details (Amount, V1, V2...).
    Output: A printable report with Flag, Score, and Reason.
    """
    input_df = pd.DataFrame([transaction_data])
    
    
    raw_score = iso_model.decision_function(input_df)[0]
    
    if raw_score > 0:
        risk_score = 100 - (raw_score * 300) 
        risk_score = max(0, risk_score) 
    else:
        risk_score = 50 + (abs(raw_score) * 200)
        risk_score = min(100, risk_score) 
        
    risk_score = round(risk_score, 2)

    if risk_score > 70:
        flag = "🔴 RED (Anomaly Detected)"
    elif risk_score > 50:
        flag = "🟡 YELLOW (Unusual Pattern)"
    else:
        flag = "🟢 GREEN (Normal)"

    avg_amount = X['Amount'].mean()
    reasons = []
    
    if transaction_data['Amount'] > avg_amount * 5:
        reasons.append(f"Amount (${transaction_data['Amount']}) is huge (5x Average).")
    
    if flag != "🟢 GREEN" and not reasons:
        reasons.append("Complex statistical anomaly in transaction metadata (V-features).")
    elif flag == "🟢 GREEN":
        reasons.append("Transaction fits normal spending profile.")

    print("-" * 40)
    print(f"TREASURY MONITOR REPORT")
    print("-" * 40)
    print(f"Amount:     ${transaction_data['Amount']}")
    print(f"RISK SCORE: {risk_score}/100")
    print(f"STATUS:     {flag}")
    print(f"REASON:     {' '.join(reasons)}")
    print("-" * 40)



normal_tx = X.iloc[0].to_dict() 
normal_tx['Amount'] = 10.0      

anomaly_tx = X.iloc[0].to_dict()
anomaly_tx['Amount'] = 25000.0 
anomaly_tx['V1'] = -50.0 

print("\n\n--- TESTING NORMAL SPENDING ---")
check_spending_risk(normal_tx)

print("\n\n--- TESTING ABNORMAL SPENDING ---")
check_spending_risk(anomaly_tx)



--- TESTING NORMAL SPENDING ---
----------------------------------------
TREASURY MONITOR REPORT
----------------------------------------
Amount:     $10.0
RISK SCORE: 83.52/100
STATUS:     🔴 RED (Anomaly Detected)
REASON:     Complex statistical anomaly in transaction metadata (V-features).
----------------------------------------


--- TESTING ABNORMAL SPENDING ---
----------------------------------------
TREASURY MONITOR REPORT
----------------------------------------
Amount:     $25000.0
RISK SCORE: 85.82/100
STATUS:     🔴 RED (Anomaly Detected)
REASON:     Amount ($25000.0) is huge (5x Average).
----------------------------------------


In [6]:
user_csv_data = {
    'TransactionID': ['TX101', 'TX102', 'TX103'],
    'Amount': [50.0, 15000.0, 20.5],
}
for i in range(1, 29):
    user_csv_data[f'V{i}'] = [0.0, -10.0, 0.1] 

pd.DataFrame(user_csv_data).to_csv('daily_spending_log.csv', index=False)
print(" User file 'daily_spending_log.csv' received.")


def generate_spending_report(filename):
    print(f"Processing {filename}...")
    df = pd.read_csv(filename)
    
    features_needed = ['Amount'] + [f'V{i}' for i in range(1, 29)]
    input_data = df[features_needed]
    
    preds = iso_model.predict(input_data)
    scores = iso_model.decision_function(input_data)
    
    results = []
    for i, pred in enumerate(preds):
        score_val = scores[i]
        
        if pred == -1:
            status = "🔴 ANOMALY"
            risk = 80 + abs(score_val * 100)
            risk = min(99.9, risk)
        else:
            status = "🟢 NORMAL"
            risk = 10 + abs(score_val * 10)
            
        results.append({
            'ID': df.iloc[i]['TransactionID'],
            'Amount': df.iloc[i]['Amount'],
            'Risk_Score': round(risk, 1),
            'Status': status
        })
        
    return pd.DataFrame(results)

report = generate_spending_report('daily_spending_log.csv')
print("\n---  FINAL TREASURY REPORT ---")
display(report)

 User file 'daily_spending_log.csv' received.
Processing daily_spending_log.csv...

---  FINAL TREASURY REPORT ---


,ID,Amount,Risk_Score,Status
0,TX101,50.0,12.1,🟢 NORMAL
1,TX102,15000.0,99.9,🔴 ANOMALY
2,TX103,20.5,12.1,🟢 NORMAL


In [7]:
import joblib

joblib.dump(iso_model, 'spending_anomaly_model.pkl')

print(" 'spending_anomaly_model.pkl' has been created.")
print("Download it from the Output folder.")

 'spending_anomaly_model.pkl' has been created.
Download it from the Output folder.
